In [ ]:
import pandas as pd
import numpy as np
import tkinter as tk
import calendar
import warnings
import locale
import os
from tkinter import messagebox
from datetime import datetime

warnings.filterwarnings('ignore')
locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')

def obtener_fecha(mes_anterior=False):
    hoy = datetime.now()
    
    if mes_anterior:
        if hoy.month == 1:
            mes = 12
            año = hoy.year - 1
        else:
            mes = hoy.month - 1
            año = hoy.year
    else:
        mes = hoy.month
        año = hoy.year
    
    mes_nombre = calendar.month_abbr[mes].upper()[:3] # ENE
    mes_año = f"{mes_nombre}{str(año)[2:]}" # ENE24
    fecha = f"{año}{str(mes).zfill(2)}" # 202401
    
    return mes_año, fecha

root = tk.Tk()
root.attributes('-topmost', True)
root.withdraw()

result = messagebox.askquestion('Confirmación', '¿Cargar mes anterior?', icon='warning')
if result == 'yes':
    mes_año, fecha = obtener_fecha(mes_anterior=True)
else:
    mes_año, fecha = obtener_fecha()
root.destroy()

efectividades_path = f'bases/efectividades/{fecha}/Efectividades_{fecha}.xlsx'
dashboard_efect_path = f'dashboard/{fecha}/DASHBOARD_EFECTIVIDADES_{mes_año}.xlsx'
dashboard_saldos_path = f'dashboard/{fecha}/DASHBOARD_SALDOS_{mes_año}.xlsx'
dashboard_indicadores_path = f'dashboard/{fecha}/DASHBOARD_INDICADORES_{mes_año}.xlsx'

dashboard_efect_path = os.path.abspath(dashboard_efect_path)
dashboard_saldos_path = os.path.abspath(dashboard_saldos_path)
dashboard_indicadores_path = os.path.abspath(dashboard_indicadores_path)

print(efectividades_path)
print(dashboard_efect_path)
print(dashboard_saldos_path)
print(dashboard_indicadores_path)

In [3]:
def clean_columns(columns_list: list[str]) -> list[str]:
    return [column.strip().replace('.', '').replace(' ', '_').upper() for column in columns_list]

In [ ]:
root = tk.Tk()
root.attributes('-topmost', True)
root.withdraw()

result = messagebox.askquestion('Confirmación', '¿Cargar efectividades?', icon='warning')
if result == 'yes':
    df_pagos = pd.read_excel(efectividades_path)
    print('Base Efectividades:', df_pagos.shape)

root.destroy()

In [5]:
df_pagos_test = df_pagos.copy()
df_pagos_test.columns = clean_columns(df_pagos_test.columns)

#mapping = {
#    'a.<=6 meses': 'a. <=6 meses',
#    'a.<=1 año': 'b. >6 meses - <=1 año',
#    'b.>6meses-<=1año': 'b. >6 meses - <=1 año',
#    'b.>1año-<=2 años': 'c. >1 - <=2 años',
#    'c.>1año-<=2 años': 'c. >1 - <=2 años',
#    'c.>2años-<=3años': 'd. >2 - <=3 años',
#    'd.>2años': 'd. >2 - <=3 años',
#    'd.>3años': 'e. >3 años'
#}

#df_pagos_test['DESC_TRAMO'] = df_pagos_test['DESC_TRAMO'].replace(mapping)

#df_pagos_test['CLAVE'] = np.where(
#    (df_pagos_test['CLAVE'] == 'KSTBC') & (df_pagos_test['DESC_TRAMO'].isin(['d. >2 - <=3 años', 'e. >3 años'])),
#    'KSTBC_2',
#    df_pagos_test['CLAVE']
#)

In [6]:
#df_pagos_test['DESC_TRAMO'].value_counts(dropna=False).sort_index()

In [7]:
#df_pagos_test['CLAVE'].value_counts(dropna=False).sort_index()

In [8]:
df_pagos_efect = df_pagos_test.copy()
df_pagos_saldos = df_pagos_test.copy()
df_pagos_indicadores = df_pagos_test.copy()

In [9]:
def dashboard_efectividades(df_pagos_efect: pd.DataFrame) -> pd.DataFrame:
    df_pagos_efect.columns = clean_columns(df_pagos_efect.columns)
    cols =  ['PERIODO', 'CLAVE', 'AGENCIA', 'CAPITALSOLES', 'PAGOEFECTTOTALSOLESAGENCIACONT']
    df_pagos_efect = df_pagos_efect[cols]
    print(df_pagos_efect.shape)
    
    df_pagos_efect.rename(columns={
        'CLAVE': 'CARTERA', 
        'CAPITALSOLES': 'CAPITAL', 
        'PAGOEFECTTOTALSOLESAGENCIACONT': 'RECUPERO', 
    }, inplace=True)
    
    df_pagos_efect = df_pagos_efect[df_pagos_efect['AGENCIA'].isin(['ASESCOM RJ', 'CLASA MORA', 'MORNESE MORA'])]
    df_pagos_efect['CAPITAL'] = df_pagos_efect['CAPITAL'].round(2)
    df_pagos_efect['RECUPERO'] = df_pagos_efect['RECUPERO'].round(2)
    
    df_pagos_efect = df_pagos_efect.groupby(['PERIODO', 'CARTERA', 'AGENCIA']).agg({'RECUPERO': 'sum', 'CAPITAL': 'sum'}).reset_index()
    
    return df_pagos_efect

In [10]:
def dashboard_saldos(df_pagos_saldos: pd.DataFrame) -> pd.DataFrame:
    df_pagos_saldos.columns = clean_columns(df_pagos_saldos.columns)
    cols =  ['PERIODO', 'CLAVE', 'AGENCIA', 'CAPITALSOLES', 'SEGMENTO_RIESGO', 'AMBITO_RCD_FINAL']
    df_pagos_saldos = df_pagos_saldos[cols]
    print(df_pagos_saldos.shape)
    
    df_pagos_saldos.rename(columns={
        'CLAVE': 'CARTERA', 
        'CAPITALSOLES': 'CAPITAL', 
        'SEGMENTO_RIESGO': 'SEGMENTO', 
        'AMBITO_RCD_FINAL': 'PRODUCTO'
    }, inplace=True)
    
    df_pagos_saldos = df_pagos_saldos[df_pagos_saldos['AGENCIA'].isin(['ASESCOM RJ', 'CLASA MORA', 'MORNESE MORA', 'SIN AGENCIA'])]
    
    df_pagos_saldos['SEGMENTO'] = df_pagos_saldos['SEGMENTO'].apply(lambda x: 'PYME / EMP MIN' if x != 'PARTICULARES' else x)
    df_pagos_saldos['PRODUCTO'] = df_pagos_saldos['PRODUCTO'].str.replace(r'\d+', '', regex=True).str.replace('.', '').str.upper()
    df_pagos_saldos['PRODUCTO'] = df_pagos_saldos['PRODUCTO'].str.strip().replace(' ', '')
    df_pagos_saldos['CAPITAL'] = df_pagos_saldos['CAPITAL'].round(2)
    
    df_pagos_saldos = df_pagos_saldos.groupby(['PERIODO', 'CARTERA', 'AGENCIA', 'SEGMENTO', 'PRODUCTO']).agg({'CAPITAL': 'sum'}).reset_index()
    
    return df_pagos_saldos

In [11]:
def dashboard_indicadores(df_pagos_indicadores: pd.DataFrame) -> pd.DataFrame:
    df_pagos_indicadores.columns = clean_columns(df_pagos_indicadores.columns)
    cols =  ['PERIODO', 'CLAVE', 'AGENCIA', 'INTENSIDAD', 'DIRECTO_CALL', 'COBERTURA', 'CONTACTO_EFECTIVO', 'TASA_CIERRE', 'PDP_CUMPLIDA']
    df_pagos_indicadores = df_pagos_indicadores[cols]
    print(df_pagos_indicadores.shape)
    
    df_pagos_indicadores.rename(columns={
        'CLAVE': 'CARTERA', 
        'INTENSIDAD': 'INTENSIDAD_TOTAL', 
        'DIRECTO_CALL': 'INTENSIDAD_DIRECTA', 
        'CONTACTO_EFECTIVO': 'CONTACTABILIDAD_DIRECTA', 
        'PDP_CUMPLIDA': 'CALIDAD_PROMESAS'
        }, inplace=True)
    
    df_pagos_indicadores = df_pagos_indicadores[df_pagos_indicadores['AGENCIA'].isin(['ASESCOM RJ', 'CLASA MORA', 'MORNESE MORA'])]
    
    df_pagos_indicadores['INTENSIDAD_TOTAL'] = df_pagos_indicadores['INTENSIDAD_TOTAL'].fillna(0)
    df_pagos_indicadores['INTENSIDAD_DIRECTA'] = df_pagos_indicadores['INTENSIDAD_DIRECTA'].fillna(0)
    df_pagos_indicadores['COBERTURA'] = df_pagos_indicadores['COBERTURA'].fillna(0)
    df_pagos_indicadores['CONTACTABILIDAD_DIRECTA'] = df_pagos_indicadores['CONTACTABILIDAD_DIRECTA'].fillna('NULL')
    df_pagos_indicadores['TASA_CIERRE'] = df_pagos_indicadores['TASA_CIERRE'].fillna('NULL')
    df_pagos_indicadores['CALIDAD_PROMESAS'] = df_pagos_indicadores['CALIDAD_PROMESAS'].fillna('NULL')
    # convierte 'NULL' en NaN
    df_pagos_indicadores['COBERTURA'] = pd.to_numeric(df_pagos_indicadores['COBERTURA'], errors='coerce')
    df_pagos_indicadores['CONTACTABILIDAD_DIRECTA'] = pd.to_numeric(df_pagos_indicadores['CONTACTABILIDAD_DIRECTA'], errors='coerce')
    df_pagos_indicadores['TASA_CIERRE'] = pd.to_numeric(df_pagos_indicadores['TASA_CIERRE'], errors='coerce')
    df_pagos_indicadores['CALIDAD_PROMESAS'] = pd.to_numeric(df_pagos_indicadores['CALIDAD_PROMESAS'], errors='coerce')
    
    df_pagos_indicadores = df_pagos_indicadores.groupby(['PERIODO', 'CARTERA', 'AGENCIA']).agg({
        'INTENSIDAD_TOTAL': 'mean', 
        'INTENSIDAD_DIRECTA': lambda x: x[x != 0].mean(), 
        'COBERTURA': 'mean', 
        'CONTACTABILIDAD_DIRECTA': lambda x: x.mean(skipna=True), 
        'TASA_CIERRE': lambda x: x.mean(skipna=True), 
        'CALIDAD_PROMESAS': lambda x: x.mean(skipna=True)
    }).reset_index()
    
    df_pagos_indicadores['INTENSIDAD_TOTAL'] = df_pagos_indicadores['INTENSIDAD_TOTAL'].round(2)
    df_pagos_indicadores['INTENSIDAD_DIRECTA'] = df_pagos_indicadores['INTENSIDAD_DIRECTA'].round(2)
    df_pagos_indicadores['COBERTURA'] = df_pagos_indicadores['COBERTURA'].round(4)
    df_pagos_indicadores['CONTACTABILIDAD_DIRECTA'] = df_pagos_indicadores['CONTACTABILIDAD_DIRECTA'].round(2)
    df_pagos_indicadores['TASA_CIERRE'] = df_pagos_indicadores['TASA_CIERRE'].round(2)
    df_pagos_indicadores['CALIDAD_PROMESAS'] = df_pagos_indicadores['CALIDAD_PROMESAS'].round(2)
    
    df_pagos_indicadores.sort_values(by=['PERIODO', 'CARTERA', 'AGENCIA'], inplace=True)
    
    return df_pagos_indicadores

In [ ]:
df_pagos_efect = dashboard_efectividades(df_pagos_efect)
df_pagos_efect.to_excel(dashboard_efect_path, index=False)
print(df_pagos_efect.shape)
df_pagos_efect.head()

In [ ]:
df_pagos_saldos = dashboard_saldos(df_pagos_saldos)
df_pagos_saldos.to_excel(dashboard_saldos_path, index=False)
print(df_pagos_saldos.shape)
df_pagos_saldos.head()

In [ ]:
df_pagos_indicadores = dashboard_indicadores(df_pagos_indicadores)
df_pagos_indicadores.to_excel(dashboard_indicadores_path, index=False)
print(df_pagos_indicadores.shape)
df_pagos_indicadores.head()

In [15]:
os.startfile(dashboard_efect_path)
os.startfile(dashboard_saldos_path)
os.startfile(dashboard_indicadores_path)